# Text Generation for Explaining Project Management

In [4]:
import json
import os
import sys
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name='us-west-2'
)

In [5]:
scenario_description = """
Explain what hamsters are to 9th graders.
"""

In [6]:
request_body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [
        {"role": "user", "content": scenario_description}
    ],
    "max_tokens": 300,
    "temperature": 0.1,
    "top_p": 0.9
})

In [7]:
modelId = 'anthropic.claude-3-5-sonnet-20241022-v2:0'
accept = 'application/json'
contentType = 'application/json'
generatedText = "\n"

model_response = bedrock_runtime.invoke_model(body=request_body, modelId=modelId, accept=accept, contentType=contentType)
response_contents = json.loads(model_response['body'].read())

generatedText = response_contents['content'][0]['text']

print(generatedText)

Here's an explanation of hamsters suitable for 9th graders:

Hamsters are small, furry rodents that have become popular household pets. They typically grow to about 2-6 inches long (depending on the species) and are known for their distinctive features like:
- Round bodies
- Short tails
- Puffy cheek pouches (which they use to store food)
- Small ears
- Short legs

The most common pet hamster species is the Syrian hamster (also called the golden hamster), but there are other types like dwarf hamsters and Roborovski hamsters.

Some key facts about hamsters:
- They're nocturnal, meaning they're most active at night
- They have a relatively short lifespan of about 2-3 years
- They're omnivores, eating both plants and insects in the wild
- As pets, they eat specially formulated hamster food, seeds, and fresh vegetables
- They're known for storing food in their cheek pouches, which can expand to carry food back to their burrows
- In the wild, they live in burrows underground
- They're gener

In [8]:
model_response

{'ResponseMetadata': {'RequestId': '810bc7da-08ca-42e0-85d4-09acab11dace',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 17 Feb 2025 10:19:37 GMT',
   'content-type': 'application/json',
   'content-length': '1493',
   'connection': 'keep-alive',
   'x-amzn-requestid': '810bc7da-08ca-42e0-85d4-09acab11dace',
   'x-amzn-bedrock-invocation-latency': '6518',
   'x-amzn-bedrock-output-token-count': '300',
   'x-amzn-bedrock-input-token-count': '22'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x193424b9d60>}

In [9]:
response_contents

{'id': 'msg_bdrk_01ALoHnevwp7oZ7D6LT6eUJV',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text',
   'text': "Here's an explanation of hamsters suitable for 9th graders:\n\nHamsters are small, furry rodents that have become popular household pets. They typically grow to about 2-6 inches long (depending on the species) and are known for their distinctive features like:\n- Round bodies\n- Short tails\n- Puffy cheek pouches (which they use to store food)\n- Small ears\n- Short legs\n\nThe most common pet hamster species is the Syrian hamster (also called the golden hamster), but there are other types like dwarf hamsters and Roborovski hamsters.\n\nSome key facts about hamsters:\n- They're nocturnal, meaning they're most active at night\n- They have a relatively short lifespan of about 2-3 years\n- They're omnivores, eating both plants and insects in the wild\n- As pets, they eat specially formulated hamster food, seeds, and fresh v

In [16]:
print(response_contents['content'][0]['text'])

Here's an explanation of hamsters suitable for 9th graders:

Hamsters are small, furry rodents that have become popular household pets. They typically grow to about 2-6 inches long (depending on the species) and are known for their distinctive features like:
- Round bodies
- Short tails
- Puffy cheek pouches (which they use to store food)
- Small ears
- Short legs

The most common pet hamster species is the Syrian hamster (also called the golden hamster), but there are other types like dwarf hamsters and Roborovski hamsters.

Some key facts about hamsters:
- They're nocturnal, meaning they're most active at night
- They have a relatively short lifespan of about 2-3 years
- They're omnivores, eating both plants and insects in the wild
- As pets, they eat specially formulated hamster food, seeds, and fresh vegetables
- They're known for storing food in their cheek pouches, which can expand to carry food back to their burrows
- In the wild, they live in burrows underground
- They're gener

# Streaming an Explanation of Hamsters

In [63]:
import json

output = []
streamed_response = bedrock_runtime.invoke_model_with_response_stream(
    body=request_body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
stream = streamed_response.get('body')

i = 1
if stream:
    try:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_data = chunk.get('bytes')
                if chunk_data:
                    decoded_chunk = chunk_data.decode()
                    chunk_obj = json.loads(decoded_chunk)

                    # Claude models return text in 'delta' -> 'text'
                    text = chunk_obj.get("delta", {}).get("text", "")

                    if text.strip():  # Only append non-empty responses
                        output.append(text)
                        print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                        i += 1

    except Exception as e:
        print(f'Error processing stream: {str(e)}')


		**Chunk 1**
Here

		**Chunk 2**
's

		**Chunk 3**
 an explanation of ham

		**Chunk 4**
sters suitable

		**Chunk 5**
 for 9th

		**Chunk 6**
 graders:

		**Chunk 7**


Hamsters are

		**Chunk 8**
 small,

		**Chunk 9**
 furry rodents

		**Chunk 10**
 that have

		**Chunk 11**
 become

		**Chunk 12**
 popular

		**Chunk 13**
 househol

		**Chunk 14**
d pets. They typically

		**Chunk 15**
 grow

		**Chunk 16**
 to about

		**Chunk 17**
 2

		**Chunk 18**
-6

		**Chunk 19**
 inches long (

		**Chunk 20**
depending

		**Chunk 21**
 on the species)

		**Chunk 22**
 and are

		**Chunk 23**
 known for their distinctive

		**Chunk 24**
 features

		**Chunk 25**
 like:

		**Chunk 26**

- Roun

		**Chunk 27**
d bodies

		**Chunk 28**

- Short

		**Chunk 29**
 t

		**Chunk 30**
ails
- Small

		**Chunk 31**
 ears
-

		**Chunk 32**
 Che

		**Chunk 33**
ek pouches that

		**Chunk 34**
 can expan

		**Chunk 35**
d to store

		**Chunk 36**
 food
- Sharp

		**Chunk 37**
 teeth that

		**Chunk 38**


In [64]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**

Here's an explanation of hamsters suitable for 9th graders:

Hamsters are small, furry rodents that have become popular household pets. They typically grow to about 2-6 inches long (depending on the species) and are known for their distinctive features like:
- Round bodies
- Short tails
- Small ears
- Cheek pouches that can expand to store food
- Sharp teeth that continuously grow

There are several common pet hamster species, including:
- Syrian hamsters (also called Golden hamsters)
- Dwarf hamsters (like Roborovski and Campbell's hamsters)

In the wild, hamsters are mostly nocturnal, meaning they're most active at night. They're naturally found in parts of Europe and Asia, where they live in burrows underground.

As pets, hamsters are relatively low-maintenance but need:
- A secure cage with bedding
- Exercise equipment like wheels
- Fresh water and food (seeds, pellets, and some fruits/vegetables)
- Regular cage cleaning
- Gentle handling

While they're cute 